# Assistant-04<br/>Function calling

## Trading Assistant with email functionality

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

## Email functionality

- Implemented using Logic Apps with an Http POST trigger

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import time
import requests
import oaihelper as framework
import yfinance as yf


## Get an OpenAI client
client = framework.get_openai_client(api_key=framework.api_KEY,
        api_version=framework.api_version,
        azure_endpoint=framework.api_URI,)

### Get the latest stock price by ticker symbol using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]    
    return price


### Use a Logic App HTTP POST trigger to email using Outlook

In [3]:
def send_email(json_payload):
    headers = {'Content-Type': 'application/json'}
    response = requests.post(framework.email_URI, json=json_payload, headers=headers)
    if response.status_code == 202:
        print("Email sent to: " + json_payload['to'])  

### Prepare the tool for function calling

In [4]:
tools_list = [{
    "type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
},
{
    "type": "function",
    "function": {

        "name": "send_email",
        "description": "Sends an email to a recipient(s).",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The email(s) the email should be sent to."
                },
                "content": {
                    "type": "string",
                    "description": "The content of the email."
                }
            },
            "required": ["to", "content"]
        }
    }
}]

### Create the assistant

In [5]:
framework.clear_shelves()

## Create an assistant
stock_assistant = framework.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list, 
                                                  model=framework.gpt_deployment_name)

Added assistant:  asst_arylhXwQihkVI27k0aX3eYo9 1


### Create a thread, put a message on the thread, and view the initial status

In [6]:
# Step 2: Create a Thread
thread = client.beta.threads.create()

article = """Title: Tech Giants Microsoft and Nvidia Outshine Others in Recent Market Performance

New York, New York 01/15/2024

Introduction: In the ever-evolving world of technology, the performance of major companies often serves as a barometer for the industry's overall health. In recent times, two tech giants, Microsoft (MSFT) and Nvidia (NVDA), have stood out from the pack with their impressive market performance. However, other prominent tickers such as Apple (AAPL), Novo Nordisk (NVO), MercadoLibre (MELI), Tradeweb Markets (TW), and Tesla (TSLA) have struggled to keep pace. Let's delve into the details of their respective performances and explore the factors contributing to their divergent trajectories.

Microsoft (MSFT): Microsoft, a global leader in software and cloud services, has been on a remarkable upward trajectory. The company's stock price has experienced consistent growth, driven by its strong financial performance and successful transition to a cloud-based business model. MSFT has demonstrated resilience during the COVID-19 pandemic, with increased demand for its cloud services and productivity tools. Additionally, Microsoft's strategic acquisitions and investments in emerging technologies have bolstered its position in the market.

Nvidia (NVDA): Nvidia, renowned for its advanced graphics processing units (GPUs) and artificial intelligence (AI) technologies, has also been a standout performer. The company's stock price has soared, fueled by its dominance in the gaming industry and its expanding presence in AI and data centers. Nvidia's GPUs have become essential components in various sectors, including gaming, autonomous vehicles, and data centers, driving its revenue growth. Furthermore, the company's recent acquisition of Arm Holdings has positioned it for further expansion into the semiconductor market.

Apple (AAPL): While Apple remains one of the most valuable companies globally, its recent performance has been relatively lackluster compared to Microsoft and Nvidia. The tech giant faced challenges due to the COVID-19 pandemic, which disrupted its supply chain and impacted iPhone sales. However, Apple's strong ecosystem, including its services segment and wearable devices, has helped mitigate some of the setbacks. The upcoming release of new products and the continued growth of its services division may provide a catalyst for Apple's future performance.

Novo Nordisk (NVO), MercadoLibre (MELI), Tradeweb Markets (TW), and Tesla (TSLA): In contrast to Microsoft and Nvidia, Novo Nordisk, MercadoLibre, Tradeweb Markets, and Tesla have faced difficulties in recent market performance.

Novo Nordisk, a leading pharmaceutical company specializing in diabetes care, has experienced challenges due to increased competition and pricing pressures in the industry. However, the company's strong pipeline of innovative products and its commitment to research and development may help it regain momentum.

MercadoLibre, often referred to as the "Amazon of Latin America," has faced headwinds due to economic uncertainties in the region. Despite being a dominant e-commerce player, the company's profitability has been impacted by increased investments in logistics and expansion into new markets. However, MercadoLibre's long-term growth prospects remain promising, given the region's growing digital economy.

Tradeweb Markets, a global operator of electronic marketplaces for fixed-income and derivatives trading, has experienced a slowdown in trading volumes due to market volatility and reduced investor activity. However, the company's strong market position and ongoing digital transformation efforts may help it navigate these challenges.

Tesla, the electric vehicle (EV) pioneer, has witnessed significant volatility in its stock price. While the company has achieved remarkable success in the EV market, concerns about valuation and competition have impacted its performance. Tesla's ability to ramp up production, expand its product lineup, and maintain its technological edge will be crucial for its future success.

Conclusion: In the world of technology, market performance can vary significantly among major players. Microsoft and Nvidia have emerged as top performers, driven by their strong financials, strategic investments, and market dominance. On the other hand, Apple, Novo Nordisk, MercadoLibre, Tradeweb Markets, and Tesla have faced challenges that have impacted their recent performance. However, each company possesses unique strengths and growth opportunities that could potentially turn the tide in their favor. Investors and industry observers will closely monitor these companies as they navigate the ever-changing landscape of the tech industry."""

# Step 3: Add a Message to a Thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="For the following article: \n\nArticle: """"""" + article + "\n""""""" +
    "\n\nEmail a summary of the article in simple HTML format, include today's date, include a list of the closing prices of the stocks mentioned in the article to: alemor@microsoft.com and vsantana@microsoft.com." +
    "Use only the provide text."
)

# Step 4: Run the Assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=stock_assistant.id,
    instructions="Today's date and time is: " + framework.get_localized_datetime() + ".\n\n"
)

#print(run.model_dump_json(indent=4))

### Process the message, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [7]:
while True:
    # Wait for 2 seconds
    time.sleep(2)

    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    #print(run_status.model_dump_json(indent=4))

    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )

        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")

        ## Cleanup
        print(client.beta.threads.delete(thread.id))
        print(client.beta.assistants.delete(stock_assistant.id))

        break
    elif run_status.status == 'requires_action':
        print("Function Calling")
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tool_outputs = []
        import json
        for action in required_actions["tool_calls"]:
            func_name = action['function']['name']
            arguments = json.loads(action['function']['arguments'])
            
            if func_name == "get_stock_price":
                output = get_stock_price(symbol=arguments['symbol'])
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": output
                })
            elif func_name == "send_email":
                print("Sending email...")
                email_to = arguments['to']
                email_content = arguments['content']
                send_email({'to': email_to,'content':email_content})
                
                tool_outputs.append({
                    "tool_call_id": action['id'],
                    "output": "Email sent"
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
        print("Submitting outputs back to the Assistant...")
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        print("Waiting for the Assistant to process...")
        time.sleep(2)

Waiting for the Assistant to process...
Waiting for the Assistant to process...
Waiting for the Assistant to process...
Function Calling
{'tool_calls': [{'id': 'call_ngZlnts0EokdtChQYojNMvSs', 'function': {'arguments': '{"symbol": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_JSdcVJKezQw0gQdBoPLsfl4H', 'function': {'arguments': '{"symbol": "NVDA"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_AuLLyG3PVNLVlYDWLsL5vlre', 'function': {'arguments': '{"symbol": "AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_6uxHtcDNCvMDQqoM3IR4cpSk', 'function': {'arguments': '{"symbol": "NVO"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_5HTRs2DxkBkcrzinPe9NWzhz', 'function': {'arguments': '{"symbol": "MELI"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_p0BzEP96c3lNc5T10mf9yyDA', 'function': {'arguments': '{"symbol": "TW"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_4m15cMODoysVW8u